# Preprocess the ENSEMBL data and prepare the protein coding promoters
## <span style="color:red;">The output bed is not sorted properly. So SORT the bed output<span style="color:red;">. 

Some confusions:
* Not sure the difference between Gene start (bp), Transcription start site (TSS) and Transcript start (bp) of the mart_export.txt file

In [1]:
import pandas as pd 
import pandas as pd
import os

# Define upstream and downstream values
upstream = 2000
downstream = 1000

# Define input and output file paths
input_file = "../Reference_like_Data/Ensembl/mart_export.txt"
output_file = os.path.dirname(input_file)+"/EnsemblePromotersUp"+str(upstream)+"_down"+str(downstream)

protin_file = '../Reference_like_Data/protein_coding_gene.txt'

# Read data into a pandas DataFrame
df = pd.read_csv(input_file, sep='\t', low_memory=False)
df.head()

,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Gene name,Transcription start site (TSS),Transcript start (bp),Transcript end (bp),Strand
0,MT,577,647,MT-TF,577,577,647,1
1,MT,648,1601,MT-RNR1,648,648,1601,1
2,MT,1602,1670,MT-TV,1602,1602,1670,1
3,MT,1671,3229,MT-RNR2,1671,1671,3229,1
4,MT,3230,3304,MT-TL1,3230,3230,3304,1


In [2]:
# Filter genes based on chromosome
valid_chromosomes = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y']
df = df[df['Chromosome/scaffold name'].isin(valid_chromosomes)]

In [3]:
df['Gene Length (bp)'] = abs(df['Gene end (bp)'] - df['Gene start (bp)'])

In [4]:
# Sort by gene length in descending order and drop duplicates keeping the longest gene
df = df.sort_values(by='Gene Length (bp)', ascending=False).drop_duplicates(subset='Gene name')

In [5]:
# Add 'chr' to the 'Chromosome/scaffold name' column
df['Chromosome/scaffold name'] = 'chr' + df['Chromosome/scaffold name'].astype(str)
df.head()

,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Gene name,Transcription start site (TSS),Transcript start (bp),Transcript end (bp),Strand,Gene Length (bp)
211376,chr16,5239802,7713340,RBFOX1,7664764,7664764,7710844,1,2473538
123082,chr7,146116002,148420998,CNTNAP2,147638809,147638809,148415779,1,2304996
134664,chr9,8314246,10613002,PTPRD,8733894,8317405,8733894,-1,2298756
150495,chrX,31097677,33339609,DMD,31266967,31126886,31266967,-1,2241932
248402,chr11,83455012,85628335,DLG2,85627749,85598657,85627749,-1,2173323


In [6]:
# Function to calculate the promoter region based on strand information
def calculate_promoter(row):
    if row['Strand'] == 1:
        promoter_start = max(1, row['Transcription start site (TSS)'] - upstream)  # Ensure promoter_start is not negative
        promoter_end = row['Transcription start site (TSS)'] + downstream
    else:
        promoter_start = row['Transcription start site (TSS)'] - downstream
        promoter_end = row['Transcription start site (TSS)'] + upstream
    return promoter_start, promoter_end

In [7]:
# Calculate promoters and add them to the DataFrame
df['Promoter Start (bp)'], df['Promoter End (bp)'] = zip(*df.apply(calculate_promoter, axis=1))
df.head()

,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Gene name,Transcription start site (TSS),Transcript start (bp),Transcript end (bp),Strand,Gene Length (bp),Promoter Start (bp),Promoter End (bp)
211376,chr16,5239802,7713340,RBFOX1,7664764,7664764,7710844,1,2473538,7662764,7665764
123082,chr7,146116002,148420998,CNTNAP2,147638809,147638809,148415779,1,2304996,147636809,147639809
134664,chr9,8314246,10613002,PTPRD,8733894,8317405,8733894,-1,2298756,8732894,8735894
150495,chrX,31097677,33339609,DMD,31266967,31126886,31266967,-1,2241932,31265967,31268967
248402,chr11,83455012,85628335,DLG2,85627749,85598657,85627749,-1,2173323,85626749,85629749


In [8]:
protein_df = pd.read_csv(protin_file,sep='\t',low_memory=False)
protein_df.head()

,hgnc_id,symbol,name,locus_group,locus_type,status,location,location_sortable,alias_symbol,alias_name,...,cd,lncrnadb,enzyme_id,intermediate_filament_db,rna_central_ids,lncipedia,gtrnadb,agr,mane_select,gencc
0,HGNC:5,A1BG,alpha-1-B glycoprotein,protein-coding gene,gene with protein product,Approved,19q13.43,19q13.43,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HGNC:5,ENST00000263100.8|NM_130786.4,NaN
1,HGNC:24086,A1CF,APOBEC1 complementation factor,protein-coding gene,gene with protein product,Approved,10q11.23,10q11.23,ACF|ASP|ACF64|ACF65|APOBEC1CF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HGNC:24086,ENST00000373997.8|NM_014576.4,NaN
2,HGNC:7,A2M,alpha-2-macroglobulin,protein-coding gene,gene with protein product,Approved,12p13.31,12p13.31,FWP007|S863-7|CPAMD5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HGNC:7,ENST00000318602.12|NM_000014.6,HGNC:7
3,HGNC:23336,A2ML1,alpha-2-macroglobulin like 1,protein-coding gene,gene with protein product,Approved,12p13.31,12p13.31,FLJ25179|p170,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HGNC:23336,ENST00000299698.12|NM_144670.6,HGNC:23336
4,HGNC:30005,A3GALT2,"alpha 1,3-galactosyltransferase 2",protein-coding gene,gene with protein product,Approved,1p35.1,01p35.1,IGBS3S|IGB3S,iGb3 synthase|isoglobotriaosylceramide synthase,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HGNC:30005,ENST00000442999.3|NM_001080438.1,NaN


In [9]:
protein_df_symbol = set(protein_df['symbol'].tolist())
len(protein_df_symbol)

19237

In [10]:
protein_promoter = df[df['Gene name'].isin(protein_df_symbol)]
print(protein_promoter.shape)
protein_promoter.head()

(19182, 11)


,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Gene name,Transcription start site (TSS),Transcript start (bp),Transcript end (bp),Strand,Gene Length (bp),Promoter Start (bp),Promoter End (bp)
211376,chr16,5239802,7713340,RBFOX1,7664764,7664764,7710844,1,2473538,7662764,7665764
123082,chr7,146116002,148420998,CNTNAP2,147638809,147638809,148415779,1,2304996,147636809,147639809
134664,chr9,8314246,10613002,PTPRD,8733894,8317405,8733894,-1,2298756,8732894,8735894
150495,chrX,31097677,33339609,DMD,31266967,31126886,31266967,-1,2241932,31265967,31268967
248402,chr11,83455012,85628335,DLG2,85627749,85598657,85627749,-1,2173323,85626749,85629749


In [11]:
sorted_df = protein_promoter.sort_values(by=['Chromosome/scaffold name', 'Gene start (bp)'], ascending=[True, True])
sorted_df.tail()

,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Gene name,Transcription start site (TSS),Transcript start (bp),Transcript end (bp),Strand,Gene Length (bp),Promoter Start (bp),Promoter End (bp)
861,chrY,24763069,24813492,DAZ3,24774548,24763757,24774548,-1,50423,24773548,24776548
773,chrY,24833843,24907040,DAZ4,24833940,24833940,24905391,1,73197,24831940,24834940
766,chrY,25030901,25062548,BPY2C,25062548,25043991,25062548,-1,31647,25061548,25064548
1564,chrY,25622117,25624902,CDY1,25622162,25622162,25624338,1,2785,25620162,25623162
1941,chrY,57067865,57130289,VAMP7,57067878,57067878,57128429,1,62424,57065878,57068878


In [12]:
bedlike = sorted_df[['Chromosome/scaffold name', 'Promoter Start (bp)','Promoter End (bp)','Gene name']]
bedlike.head()

,Chromosome/scaffold name,Promoter Start (bp),Promoter End (bp),Gene name
244461,chr1,63419,66419,OR4F5
273427,chr1,450678,453678,OR4F29
273754,chr1,685654,688654,OR4F16
232770,chr1,921923,924923,SAMD11
226778,chr1,958256,961256,NOC2L


In [13]:
sorted_df.to_csv(output_file+".txt", sep='\t',  index=False)
bedlike.to_csv(output_file+".bed", sep='\t', header=False, index=False)